# Thermal Neural Networks (Tensorflow example)

This jupyter notebook showcases how to utilize a [thermal neural network (TNN)](https://www.sciencedirect.com/science/article/pii/S0952197622005279) on an exemplary data set with the [Tensorflow](https://www.tensorflow.org/) framework.

This example is concise for the sake of comprehensibility, that is, no cross-validation with a validation set is conducted, e.g., for early stopping, no learning rate scheduling, no repeated experiments with different random number generator seeds, etc.

Feel free to build and expand your training pipeline on top of this example.

The data set can be downloaded from [Kaggle](https://www.kaggle.com/wkirgsn/electric-motor-temperature).
It should be placed in `data/input/`.

In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

import tensorflow as tf

## Data setup

In [ ]:
path_to_csv = Path().cwd() / "data" / "input" / "measures_v2.csv"
data = pd.read_csv(path_to_csv)
target_cols = ["pm", "stator_yoke", "stator_tooth", "stator_winding"]

temperature_cols = target_cols + ["ambient", "coolant"]
test_profiles = [60, 62, 74]
train_profiles = [p for p in data.profile_id.unique() if p not in test_profiles]
profile_sizes = data.groupby("profile_id").agg("size")

# normalize
non_temperature_cols = [c for c in data if c not in temperature_cols + ["profile_id"]]
data.loc[:, temperature_cols] /= 200  # deg C
data.loc[:, non_temperature_cols] /= data.loc[:, non_temperature_cols].abs().max(axis=0)

# extra feats (FE)
if {"i_d", "i_q", "u_d", "u_q"}.issubset(set(data.columns.tolist())):
    extra_feats = {
        "i_s": lambda x: np.sqrt((x["i_d"] ** 2 + x["i_q"] ** 2)),
        "u_s": lambda x: np.sqrt((x["u_d"] ** 2 + x["u_q"] ** 2)),
    }
data = data.assign(**extra_feats)
input_cols = [c for c in data.columns if c not in target_cols]


In [ ]:
# Rearrange features
input_cols = [c for c in data.columns if c not in target_cols + ["profile_id"]]
data = data.loc[:, input_cols + ["profile_id"] + target_cols]


def generate_tensor(profiles_list):
    tensor = np.full(
        (profile_sizes[profiles_list].max(), len(profiles_list), data.shape[1] - 1),
        np.nan,
    )
    for i, (pid, df) in enumerate(
        data.loc[data.profile_id.isin(profiles_list), :].groupby("profile_id")
    ):
        assert pid in profiles_list, f"PID is not in {profiles_list}!"
        tensor[: len(df), i, :] = df.drop(columns="profile_id").to_numpy()
    sample_weights = 1 - np.isnan(tensor[:, :, 0])
    tensor = np.nan_to_num(tensor).astype(np.float32)
    tensor = tf.convert_to_tensor(tensor)
    sample_weights = tf.convert_to_tensor(sample_weights)
    return tensor, sample_weights


train_tensor, train_sample_weights = generate_tensor(train_profiles)
test_tensor, test_sample_weights = generate_tensor(test_profiles)